<a href="https://colab.research.google.com/github/fibonacci-2/masterout/blob/main/masterout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install the transformers library
!pip install transformers

# Import necessary libraries
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch
from google.colab import drive
drive.mount('/content/drive')

# Specify the path to your model checkpoints in Google Drive
model_path = '/content/drive/MyDrive/masterout/depression_roberta-base/checkpoint-20094'

# Load the model and tokenizer for sequence classification
try:
    model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=2)
    tokenizer = RobertaTokenizer.from_pretrained(model_path)
    print("RoBERTa base model for sequence classification loaded successfully.")
except Exception as e:
    print(f"An error occurred while loading the model or tokenizer: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
RoBERTa base model for sequence classification loaded successfully.


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
import os

# Load the dataset
df = pd.read_csv('/content/drive/My Drive/amhd-copy/data/cleaned_arabic_datasetv2.csv')

# Filter for 'ocd' and 'control' classes
df_binary = df[df['condition'].isin(['depression', 'control'])].copy()

# Balance the dataset
ocd_count = df_binary[df_binary['condition'] == 'depression'].shape[0]
control_df = df_binary[df_binary['condition'] == 'control']
sampled_control_df = control_df.sample(n=ocd_count, random_state=42)

balanced_df = pd.concat([df_binary[df_binary['condition'] == 'depression'], sampled_control_df])

# Shuffle the balanced dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


# Map 'ocd' to 1 and 'control' to 0
balanced_df['label'] = balanced_df['condition'].apply(lambda x: 1 if x == 'ocd' else 0)

# Handle potential missing values in 'selftext' and 'title'
balanced_df['selftext'] = balanced_df['selftext'].fillna('')
balanced_df['title'] = balanced_df['title'].fillna('')

# Combine 'title' and 'selftext' for classification
balanced_df['text'] = balanced_df['title'] + " " + balanced_df['selftext']

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(balanced_df, test_size=0.2, random_state=42)

# Use the tokenizer loaded from the previous cell
# tokenizer is already loaded in cell 45ykJe8vXw61

# Create a custom dataset class
class OCDDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tokenize the text data using the tokenizer from the previous cell
train_encodings = tokenizer(list(train_df['text']), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_df['text']), truncation=True, padding=True, max_length=512)

# Create dataset objects
train_dataset = OCDDataset(train_encodings, list(train_df['label']))
val_dataset = OCDDataset(val_encodings, list(val_df['label']))

print("Binary classification setup complete. train_dataset and val_dataset are ready.")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Load the RoBERTa model for sequence classification
# Use the base model loaded in the previous cell and adapt it for sequence classification
# model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/masterout/ptsd_roberta-base', num_labels=2)


# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_strategy="epoch"
)

# Create Trainer instance
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

print("Trainer instance created. Ready to train the model.")

Binary classification setup complete. train_dataset and val_dataset are ready.
Training dataset size: 33736
Validation dataset size: 8434
Trainer instance created. Ready to train the model.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
